In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/hpc/compgen/users/cchang/Projects/gene_exp_VAE/data/SyNet_Normalized_Expression_Batchcorrected_Labeled_Filtered_Data_Only.csv", index_col=0)
print(df.head())


/tmp/ipykernel_17998/2771396363.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/hpc/compgen/users/cchang/Projects/gene_exp_VAE/data/SyNet_Normalized_Expression_Batchcorrected_Labeled_Filtered_Data_Only.csv", index_col=0)


   GSM107072;Pawitan  GSM107073;Pawitan  GSM107074;Pawitan  GSM107075;Pawitan  \
0           3.629049           5.280801           5.242942           4.587148   
1           4.332263           4.463688           3.707902           3.915789   
2           5.242035           4.169501           4.486855           4.602680   
3           4.884504           3.731322           3.508488           4.501715   
4           4.219822           4.804504           3.966309           3.728912   

   GSM107076;Pawitan  GSM107077;Pawitan  GSM107078;Pawitan  GSM107079;Pawitan  \
0           4.157379           4.954873           4.576248           4.935607   
1           4.222221           4.147906           4.513669           4.028902   
2           4.063071           4.252299           3.543621           5.058134   
3           4.201238           3.440060           4.393966           5.469683   
4           3.943046           4.062714           4.670924           4.612529   

   GSM107080;Pawitan  GSM1

In [4]:
df = df.iloc[:-1, :]
print(df.tail())

       GSM107072;Pawitan  GSM107073;Pawitan  GSM107074;Pawitan  \
11743           5.114423           4.500965           4.126470   
11744           4.181805           5.370797           4.975893   
11745           4.164249           3.530899           4.080236   
11746           4.194224           3.458629           4.035481   
11747           4.762019           6.531339           3.983910   

       GSM107075;Pawitan  GSM107076;Pawitan  GSM107077;Pawitan  \
11743           3.129959           4.164182           4.310675   
11744           4.656618           5.065881           5.042039   
11745           4.360459           4.060047           3.245546   
11746           5.151594           3.714482           3.946568   
11747           4.087891           4.630151           4.830090   

       GSM107078;Pawitan  GSM107079;Pawitan  GSM107080;Pawitan  \
11743           4.441228           4.755584           5.204846   
11744           4.745696           4.369580           4.787775   
11745   

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
df_scaled = scaler.fit_transform(df)
print(df_scaled)
# See https://stackoverflow.com/questions/61032867/how-to-scale-data-between-1-and-1-in-pandas. 

[[-0.60251665  0.30339016  0.28033907 ... -0.37786997 -0.2982342
  -0.42050508]
 [-0.21714469 -0.14626653 -0.56461446 ... -0.38583091 -0.30740355
   0.38552677]
 [ 0.2814234  -0.30815687 -0.1358441  ... -0.43625668  0.23093531
  -0.27111731]
 ...
 [-0.30921923 -0.65957925 -0.35966525 ...  0.21093143  0.05654576
  -0.47280828]
 [-0.29279227 -0.69934924 -0.38430053 ...  0.14698923 -0.32670686
   0.04930219]
 [ 0.01836773  0.99156014 -0.41268744 ...  0.38457623  0.07521055
   0.50098755]]


In [23]:
# Create new df for batch labels
import string

id_list = []
study_list = []
split_label_list = []

# Take the head of the df
sample_header_list = list(df)


for name in sample_header_list:
    split = name.split(';') 
    id_list.append(split[0])
    study_list.append(split[1])
    if split[1] == 'TCGA':
        split_label_list.append('val')
    elif split[1] == 'Miller' or split[1] == 'Minn':
        split_label_list.append('test')
    else:
        split_label_list.append('train')
        
study_name_list = set(study_list)
batch_label_list = list(string.ascii_uppercase[:len(study_name_list)])
study_name_dict = dict(zip(study_name_list, batch_label_list))

if len(id_list) == len(study_list) == len(split_label_list):
    df_batch_label = pd.DataFrame(list(zip(id_list, study_list, split_label_list)), columns=['sample_id', 'study_name', 'split_label'])

df_batch_label['batch_label'] = df_batch_label['study_name'].apply (lambda x: study_name_dict.get(x))

print(df_batch_label)

            sample_id study_name split_label batch_label
0           GSM107072    Pawitan       train           E
1           GSM107073    Pawitan       train           E
2           GSM107074    Pawitan       train           E
3           GSM107075    Pawitan       train           E
4           GSM107076    Pawitan       train           E
...               ...        ...         ...         ...
3965  TCGA-E2-A108-01       TCGA         val           L
3966  TCGA-BH-A0B2-01       TCGA         val           L
3967  TCGA-AO-A12B-01       TCGA         val           L
3968  TCGA-E2-A109-01       TCGA         val           L
3969  TCGA-AO-A128-01       TCGA         val           L

[3970 rows x 4 columns]
